In [61]:
from aima.agents import Environment, Thing, Agent
from collections import namedtuple
from random import randrange


Position = namedtuple("Position", "x y")


class AlphaEnvironment(Environment):
    def __init__(self, map_items, height=6, width=6, agent=None):
        super().__init__()
        self.width = width
        self.height = height
        self.matrix = [[None for _ in range(height)] for _ in range(width)]
        self.agents.append(agent)
        for thing in map_items:
            self.matrix[thing["x"]][thing["y"]] = thing["item"] 
        self.print_board()
          
    def percept(self, agent):
        def get_item(x, y):
            try:
                if x < 0 or y < 0:
                    return Outside()
                return self.matrix[x][y]
            except IndexError:
                return Outside()
                
        x = agent.x
        y = agent.y
        return [self.matrix[x][y], 
                get_item(x - 1, y),
                get_item(x + 1, y),
                get_item(x, y - 1),
                get_item(x, y + 1), ]

    def execute_action(self, agent, action):
        pass
    
    def print_board(self):
        for row in zip(*self.matrix):
            print("-" * (len(row) * 2 + 1))
            for item in row:
                if isinstance(item, Thing):
                    label = item.label()
                else:
                    label = "-"
                
                print("|{}".format(label), end="")
            print("|")
        print("-" * (len(self.matrix[0]) * 2 + 1))

    def step(self):
        super().step()
    

class MyThing(Thing):
    def value(self):
        raise NotImplementedError
    
    def label(self):
        raise NotImplementedError


class Outside(Thing):
    def label(self):
        return "O"


class Idol(MyThing):
    def value(self):
        return 10

    def label(self):
        return "I"


class ArrowTrap(MyThing):
    def value(self):
        return -5

    def label(self):
        return "T"


class Wall(Thing):
    def label(self):
        return "W"


class Diamond(MyThing):
    def value(self):
        return 20

    def label(self):
        return "I"


class Explorer(Agent):
    def __init__(self, position=None, program=None):
        super().__init__(program)
        if position is None:
            self.x = 0
            self.y = 0
        else:
            self.x = position.x
            self.y = position.y
            
        self.performance = 0
 
    def move(self, direction):
        self.performance -= 1
        if direction == "Up":
            self.y -= 1
        if direction == "Down":
            self.y += 1
        if direction == "Right":
            self.x += 1
        if direction == "Left":
            self.x -= 1
    
    def take(self):
        self.performance += 10


def program(percepts):
    options = []
    for i, percept in enumerate(percepts, 0):
        if isinstance(percept, Wall) and i == 0:
            return "NoOp"
        if isinstance(percept, Outside):
            options.append({"action": choose_action(i), "value": -11})
        if isinstance(percept, MyThing):
            options.append({"action": choose_action(i), "value": percept.value() - 1})
        elif percept is None and i != 0:
            options.append({"action": choose_action(i), "value": -1})
    
    if len(options) > 0:
        return max(options, key=lambda item: item["value"])["action"]
    
    else:
        return "NoOp"


def choose_action(index):
    if index == 0:
        return "Take"
    elif index == 1:
        return "MoveLeft"
    elif index == 2:
        return "MoveRight"
    elif index == 3:
        return "MoveUp"
    elif index == 4:
        return "MoveDown"


items = [{"x": 1, "y": 1, "item": Idol()}, 
         {"x": 4, "y": 5, "item": ArrowTrap()}]
a = AlphaEnvironment(items, height=7)
exp = Explorer(Position(4, 6))

print(a.percept(exp))
p = [None, ArrowTrap(), Outside(), Outside(), ArrowTrap()]
print(program(p))

-------------
|-|-|-|-|-|-|
-------------
|-|I|-|-|-|-|
-------------
|-|-|-|-|-|-|
-------------
|-|-|-|-|-|-|
-------------
|-|-|-|-|-|-|
-------------
|-|-|-|-|T|-|
-------------
|-|-|-|-|-|-|
---------------
[None, None, None, <ArrowTrap>, <Outside>]
MoveLeft
